In [81]:
#!pip install google-cloud-bigquery==2.34.3
#!pip install pandas-gbq
#!pip install db-dtypes

In [86]:
# modules
import pandas as pd
import db_dtypes
from google.cloud import bigquery
import pandas_gbq

In [87]:
# google-cloud-bigquery

# connections
gcp_project = 'king-ds-recruit-candidate-635'
bq_dataset = 'abtest'
client = bigquery.Client(project=gcp_project)
dataset_ref = client.dataset(bq_dataset)

# gcp to df function
def gcp2df(query): 
    query_job = client.query(query)  # API request
    results = query_job.result()
    return results.to_dataframe()

In [ ]:
# pandas_gbq

def gcp2df(query):
    return pd.read_gbq(query, project_id=gcp_project, dialect='standard')

In [88]:
query = ('''
    SELECT *
    FROM `king-ds-recruit-candidate-635.abtest.activity`
    LIMIT 100
    ''')

df = gcp2df(query)

ValueError: Please install the 'db-dtypes' package to use this function.

In [78]:
df.to_dataframe()

ValueError: Please install the 'db-dtypes' package to use this function.